In [2]:
using Pkg; Pkg.activate("../.")
using LinearAlgebra 
using Revise 
using TOML
using DelimitedFiles
using Pkg 
using RigidBodyDynamics
using ForwardDiff
using SparseArrays
using Rotations
using QuadrupedBalance
const QB = QuadrupedBalance

  Activating environment at `~/dev/16715-dynamics/QuadrupedBalance/Project.toml`


QuadrupedBalance

In [3]:
#### Instantiating a Quadruped model instance
## This loads in and create a RigidbodyDynamics mechanism instance, and we then passed it in 
## to UnitreeA1FullBody in QuadrupedBalance. Full detail in src/rigidbodymodel.jl and src/quadruped_fullbody.jl

urdfpath = joinpath(@__DIR__, "..", "src","a1","urdf","a1.urdf")
A1mech = parse_urdf(urdfpath, floating=true, remove_fixed_tree_joints=false)
A1 = QuadrupedBalance.UnitreeA1FullBody(A1mech);

In [8]:
#### Floating base Dynamics 
# load example eq point 
data = TOML.parsefile("ipopt_eq_point.toml") # load eq point 
x_eq = data["x_eq"];
u_eq = data["u_eq"];
λ_eq = data["λ_eq"];

ẋ = QB.dynamics(A1, x_eq, u_eq);

In [16]:
#### state definition
"""
    Full body quadruped model and state definition
    x = [q; v]
    q  = [attitude  [4 x 1]
          position  [3 x 1]
          hip_angle [4 x 1]
          thigh_angle [4 x 1]
          calf_angle] [4 x 1]
    
    v = [ang_vel  [3 x 1]
         v_trans (body frame) [3 x 1]
         hip_angle [4 x 1]
         thigh_angle [4 x 1]
         calf_angle] [4 x 1]
"""

quat = x_eq[1:4]  # attitude
r = x_eq[5:7]  # position
q = x_eq[8:19] # joint angles.
qv = x_eq[26:end] # joint vel
ω = x_eq[20:22]
v = x_eq[23:25]

### Foward kinematics to the foot
p = QB.fk(q) # body frame
p_world = QB.fk_world(x_eq) # world frame

### Kinematic Jacobian
J = QB.dfk(q)
J_world = QB.dfk_world(x_eq);

In [24]:
### Constrained Dynamics 
# QB.pinned_dynamics (defined in src/quadruped_fullbody.jl) takes in x, u, λ (ground reaction forces)
# and foot indices (foot positions that we want to constrain the robot to. In this case, we're constraining
# FR, RL foot, therefore 1:3 and 10:12 in the fk vector)

foot_indices = [1:3 ; 10:12]
QB.pinned_dynamics(A1, x_eq, u_eq, λ_eq, foot_indices) # should be zeros at eq point

### Dynamics jacobian
# defined in utils.jl, this function diff thru pinned_dynamics and return jacobian w.r.t 
# x, u, and \lambda
A, B, C = QB.dynamics_jacobians(A1, x_eq, u_eq, λ_eq, foot_indices);